In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR, Float, Integer
import pymysql
import plotly.express as px
import plotly as py
import plotly.graph_objs as go

In [8]:
def bond_type_std(bond_name,bond_type):
    '''
    func:根据债券名称和windtype对债券类型进行细化
    :param: bond_name 原有债券名 df
    :return: std_bond_type标准化的债券品种 格式list
    '''
    std_list = ['国开','农发','口行','国债','地方债','同业存单','短融','一般中期票据','超短期融资债','企业债','其他'] 
    std_bond_type=[]
    #首先对政策银行债进行细分
    for each in bond_name:
        if each is None:
            std_bond_type.append('')
        elif '国开' in each:
            std_bond_type.append('国开债')
        elif '农发' in each:
            std_bond_type.append('农发债')
        elif '进出' in each:
            std_bond_type.append('口行债')
        else:
            std_bond_type.append('')
    #其次，将剩下的品种分为国债、地方债及其他
    for i in range(len(bond_type)):
        if bond_type[i] == '国债':
            std_bond_type[i] = '国债'
        elif bond_type[i] == '地方政府债':
            std_bond_type[i] = '地方债'
        elif bond_type[i] == '同业存单':
            std_bond_type[i] = '同业存单'
        elif bond_type[i] == '超短期融资债券':
            std_bond_type[i] = '超短融'    
        elif bond_type[i] == '一般中期票据':
            std_bond_type[i] = '中票'
        elif bond_type[i] == '一般短期融资券':
            std_bond_type[i] = '短融'
        elif bond_type[i] == '一般企业债':
            std_bond_type[i] = '企业债'
        elif std_bond_type[i] == '':
            std_bond_type[i] = '其他'
            
    return std_bond_type



def get_data_by_group(data_df):
    '''
    先按照日期分组获得所有的每日数据表，然后遍历所有的每日数据，分别按照品种和期限分组,并计算每组成交量之和
    :param:data 绘图数据
    :return: result处理后的数据表 格式df
    '''
    data_new = data_df.groupby(['date'],as_index=False)#按日期分组
    frame = [] #存储每一日数据的df表
    #遍历所有的每日数据，分别按照品种和期限分组,并计算每组成交量之和
    for name,group in data_new:
        data_new_new = group.groupby(['bond_type','term_cat'],as_index=False).sum()
        date = str(name)
        data_new_new['date'] = date#标识该表的日期
        frame.append(data_new_new)#存储处理好的表
    result = pd.concat(frame)#合并
    return result


def DrawBubble(data):
    '''
    绘制成交量热度图：以气泡大小作为交易规模，横轴为期限（分为0-1,1-3,3-5,5-7,7-10,10+），纵轴依次往上分别是债券品种
    点击开始即可开到成交量动态变化图，也可在时间轴上随意拖动 选择当日的成交量情况图
    :param:data 绘图数据
    :return: figure成交量热度图
    '''
    type_list = ['国开债','农发债','口行债','国债','地方债','同业存单','短融','中票','超短融','企业债','其他']
    term = ['0-1','1-3','3-5','5-7','7-10','10+']
    order = {'bond_type':type_list,'term_cat':term}
    figure = px.scatter(data, x='term_cat', y='bond_type',
           color='bond_type', size='volume', size_max=60,
           animation_frame='date', animation_group='term_cat',
           range_x = [-1,7],category_orders=order,height=800,
           labels = {'term_cat':'期限', 'bond_type':'品种'})
    return figure
    #参数说明:
    # color='bond_type' 将同一种债券标示为同一种颜色
    # animation_frame: 气泡根据什么动, 这里就是根据时间
    # animation_group: 这个可以理解为我们的主体是什么, 这里是期限
    # range_x: x轴的范围，即期限范围

def draw_timeseries_by_group(each_type):
    '''
    绘制各品种成交量时间序列图：以柱状作为交易规模，横轴为期限），纵轴为成交量
    :param:data 绘图数据
    :return: figure 各品种成交量时间序列图
    适合同业存单、短融、超短融等（由于国开债、农发债等每天的交易量相差较大，且期限分类较多，
    所以如果以这种形式的图展现的话，需要不断放大才能看见柱子，否则正常大小看起来是空白）
    '''
    all_trace=[]
    data_type = data[data['bond_type'] == each_type]
    num_term = set(data_type['term_cat'])
    for each_term in num_term:
        data_yaxis = data_type[data_type['term_cat']==each_term]['volume']
        trace = go.Bar(
            x = data_type[data_type['term_cat']==each_term]['date'] ,
            y = data_yaxis ,
            name = "%s %s"%(each_type,each_term) )
        all_trace.append(trace)#将同一品种的不同期限放在一个数据集中
    layout = go.Layout(title = '%s成交量时间序列图'%each_type, yaxis=dict(title='成交量'),barmode='stack')
    figure = go.Figure(data = all_trace, layout = layout)
    return figure

def draw_period_by_group(each_type,start_time,end_time):
    '''
    绘制某一段时间内，各品种成交量时间序列图：以柱状作为交易规模，横轴为期限），纵轴为成交量
    适合所有品种绘图，推荐使用此函数绘制国开债、农发债、口行债、国债、企业债等
    :param:data 绘图数据
    :param:start_time 查询的开始时间
    :param:end_time 查询的结束时间
    :return: figure 各品种成交量时间序列图
    '''
    all_trace=[]
    data_type_period = data[(data['bond_type'] == each_type)&(data['date']>=start_time)&(data['date']<=end_time)]
    num_term = set(data_type_period['term_cat'])
    for each_term in num_term:
        trace = go.Bar(
            x = data_type_period[data_type_period['term_cat']==each_term]['date'] ,
            y = data_type_period[data_type_period['term_cat']==each_term]['volume'] ,
            name = "%s %s"%(each_type,each_term) )
        all_trace.append(trace)#将同一品种的不同期限放在一个数据集中
    layout = go.Layout(title = '%s成交量时间序列图'%each_type, yaxis=dict(title='成交量'))
    figure = go.Figure(data = all_trace, layout = layout)
    return figure

In [3]:
if __name__=='__main__':
    '''
    从sql获取数据，并转化成df格式
    '''
    #数据库参数
    user = 'root'
    password = 'Welcome123'
    db_ip = "cdb-3gsotx9q.cd.tencentcdb.com"
    db_name = 'guanc'
    port = 10059
    #连接数据库，执行查询调入数据
    engine = create_engine('mysql+pymysql://' + user + ":" + password + "@" + db_ip + ":" + str(port) + "/" + db_name,
                           encoding='utf-8')
    data_sql = "select 日期,code,name,term_cat,volume,windtype from transaction_bond "
    data = engine.execute(data_sql).fetchall()
    
    #将数据转化成dataframe格式
    data_df = pd.DataFrame(data, columns=['date', 'code', 'name','term_cat','volume','type'])
    
    #债券品种细分
    data_df['bond_type'] = bond_type_std(data_df['name'],data_df['type']) 
    
    #数据处理，获得按照品种和期限分组的每日成交量总表
    data = get_data_by_group(data_df)

In [6]:
# 绘图 
DrawBubble(data)

整体上，每天同业存单的交易量最大，其次是7-10期国开债；从时间上看，4.26，5.09，6.28时所有品种的交易量大幅骤减，然后迅速恢复（非银调休日)

国开债：分期限看，7-10年期国开债交易量最多，1-3&3-5年期次之（两者基本持平），10+交易量最少；从时间上看，相比于后期3月下旬10+国开债交易量较多（受疫情影响）

农发：分期限看，0-1&1-3年期农发债交易量波动较大，10+最少，5-7期倒二

口行：7月中旬之前 大多数时间3-5年期口行债交易量最多，6月后0-1&1-3年期的交易量有较大幅度的上升

国债：3月初 1-3,3-5,7-10年期的交易量较多，随后1-3,3-5年期的交易量呈现波动中降低趋势，9月整体交易量呈现增长趋势

地方债：期限上，3-5年期交易量最大；时间上，6月下旬至8月中旬地方债的交易量整体大幅下降，随后交易量有所恢复

同业存单：整体比较平稳，6.05-6.09交易量较大，9.21交易量也较大

中票：1-3,3-5年期交易量较多，5-7年期基本没有交易（因为基本也没有什么发行量）

企业债：整体交易量偏小，交易主要集中在1-3&3-5等期限，长期限债基本没有交易量

In [9]:
#绘制各品种成交情况的时间序列子图
std_list = ['国开债','农发债','口行债','国债','地方债','同业存单','短融','中票','超短融','企业债','其他']
draw_timeseries_by_group(std_list[0])

国开债：3-4月以及6月中旬交易量较大，6月之后呈现整体下降趋势

In [10]:
draw_timeseries_by_group(std_list[1])

农发债：4月开始呈现下降趋势，5月初有所上升

In [11]:
draw_timeseries_by_group(std_list[2])

口行债：4月开始呈现下降趋势，6月初有所上升，7月整体上交易量达到最高（主要靠0-1年期&10+期限的支撑）

In [12]:
draw_timeseries_by_group(std_list[3])

国债：4月开始呈现上升趋势，5月中旬上交易量达到最高，然后交易量逐渐下降

In [13]:
draw_timeseries_by_group(std_list[4])

In [14]:
draw_timeseries_by_group(std_list[5])

同业：大部分维持在1000左右，6月份整体交易量比较高
注：同业存单的成交量，一方面和机构需求有关系，另外也和一级发行有关系，6月份是一级发行比较低的时候，银行不适应新利率使劲抵抗，所以大家都二级买

In [15]:
draw_timeseries_by_group(std_list[6])

短融：6、9月份交易量特别高

In [16]:
draw_timeseries_by_group(std_list[7])

中票：4个波峰-4月中旬呈现下降趋势，5月到达低谷；5月中旬上升，6月初达到最高，然后继续下降，7月初又呈现上升趋势，然后交易量下降，近期有回升趋势

In [17]:
draw_timeseries_by_group(std_list[8])

超短融：3-4月份整体交易量较多